In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [2]:
Data=pd.read_csv('Kaggle_MissedAppointments.csv')
Data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [4]:
#Fixing column names and spelling for easy utilization
Data=Data.rename(columns={'Hipertension': 'Hypertension',
                        'Handcap': 'Handicap',
                        'SMS_received': 'SMSRecieved',
                        'No-show': 'NoShow'})

In [5]:
#Changed Scheduled Day and Appointment Day to a useable value. 
Data['ScheduledDay'] = pd.to_datetime(Data['ScheduledDay']).dt.date
Data['AppointmentDay'] = pd.to_datetime(Data['AppointmentDay']).dt.date
Data['DaysWaiting'] = (Data['AppointmentDay']-Data['ScheduledDay']).dt.days
Data=Data.drop(['AppointmentDay','ScheduledDay'],axis=1)

In [6]:
#Checking range of ages and most common
Data['Age'].value_counts()

 0      3539
 1      2273
 52     1746
 49     1652
 53     1651
        ... 
 115       5
 100       4
 102       2
 99        1
-1         1
Name: Age, Length: 104, dtype: int64

In [7]:
#Got rid of outliers in Age
Data=Data[(Data['Age']<100) & (Data['Age']>=0)]
Data.head()

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSRecieved,NoShow,DaysWaiting
0,2.987250e+13,5642903,F,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0
1,5.589978e+14,5642503,M,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0
2,4.262962e+12,5642549,F,62,MATA DA PRAIA,0,0,0,0,0,0,No,0
3,8.679512e+11,5642828,F,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0
4,8.841186e+12,5642494,F,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0


In [8]:
#Data Set information claims Handicap values only True/False
Data['Handicap'].value_counts()

0    108280
1      2036
2       183
3        13
4         3
Name: Handicap, dtype: int64

In [9]:
#Erase non-sense values. 
Data=Data[(Data['Handicap']<=1)]
Data['Handicap'].value_counts()

0    108280
1      2036
Name: Handicap, dtype: int64

In [10]:
#Change 0/1's to values that could be used for easier use in Association Rule Mining
Data.loc[Data['Scholarship'] == 0, 'Scholarship'] = 'NoScholarship'
Data.loc[Data['Scholarship'] == 1, 'Scholarship'] = 'Scholarship'

Data.loc[Data['Hypertension'] == 0, 'Hypertension'] = 'NoHypertension'
Data.loc[Data['Hypertension'] == 1, 'Hypertension'] = 'Hypertension'

Data.loc[Data['Diabetes'] == 0, 'Diabetes'] = 'NoDiabetes'
Data.loc[Data['Diabetes'] == 1, 'Diabetes'] = 'Diabetes'

Data.loc[Data['Alcoholism'] == 0, 'Alcoholism'] = 'NoAlcoholism'
Data.loc[Data['Alcoholism'] == 1, 'Alcoholism'] = 'Alcoholism'

Data.loc[Data['Handicap'] == 0, 'Handicap'] = 'NoHandicap'
Data.loc[Data['Handicap'] == 1, 'Handicap'] = 'Handicap'

Data.loc[Data['SMSRecieved'] == 0, 'SMSRecieved'] = 'NoSMSRecieved'
Data.loc[Data['SMSRecieved'] == 1, 'SMSRecieved'] = 'SMSRecieved'

Data.loc[Data['NoShow'] == 'No', 'NoShow'] = 'Show'
Data.loc[Data['NoShow'] == 'Yes', 'NoShow'] = 'NoShow'

In [11]:
Data.head()

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSRecieved,NoShow,DaysWaiting
0,2.987250e+13,5642903,F,62,JARDIM DA PENHA,NoScholarship,Hypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
1,5.589978e+14,5642503,M,56,JARDIM DA PENHA,NoScholarship,NoHypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
2,4.262962e+12,5642549,F,62,MATA DA PRAIA,NoScholarship,NoHypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
3,8.679512e+11,5642828,F,8,PONTAL DE CAMBURI,NoScholarship,NoHypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
4,8.841186e+12,5642494,F,56,JARDIM DA PENHA,NoScholarship,Hypertension,Diabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0


In [12]:
#Changing Age to ranges so for easier association formations
for i in range(0,100,5):
    Data['Age'] = np.where(Data['Age'].between(i,i+4),i,Data['Age'])

In [13]:
#Change Ages to the Range
s= '-'
for i in range (0,100,5):
    Data.loc[Data['Age'] == i, 'Age'] = (str(i)+s+str(i+5))

In [14]:
#Figure out longest someone spent waiting
Data['DaysWaiting'].max()

179

In [15]:
Data=Data.drop(['PatientId','AppointmentID'],axis=1)
Data.head()

,Gender,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSRecieved,NoShow,DaysWaiting
0,F,60-65,JARDIM DA PENHA,NoScholarship,Hypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
1,M,55-60,JARDIM DA PENHA,NoScholarship,NoHypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
2,F,60-65,MATA DA PRAIA,NoScholarship,NoHypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
3,F,5-10,PONTAL DE CAMBURI,NoScholarship,NoHypertension,NoDiabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0
4,F,55-60,JARDIM DA PENHA,NoScholarship,Hypertension,Diabetes,NoAlcoholism,NoHandicap,NoSMSRecieved,Show,0


In [16]:
#Change to weeks for easier association formations. New category every 7 days.
for i in range(0,27):
    Data['DaysWaiting'] = np.where(Data['DaysWaiting'].between((7*i+1),(7*i+7)),i,Data['DaysWaiting'])

In [17]:
#Separating Show and NoShow data sets
NoShow=Data[Data['NoShow'] == 'NoShow']
Show=Data[Data['NoShow'] == 'Show']

In [18]:
#Rename Column names to fit new information
Data=Data.rename(columns={'DaysWaiting': 'WeeksWaiting',
                        'Age': 'AgeRange'})

# Start of Assocation

In [19]:
#Turn all the DataSets into Arrays
NoShowList= NoShow.values.tolist()
NoShowAr=np.array(NoShowList)

ShowList= Show.values.tolist()
ShowAr=np.array(ShowList)

All=Data.values.tolist()
AllAr=np.array(All)

### Creating the functions to find Associations

In [20]:
#Equations for frequency of Value in the Data Set
def FindFrequency (DataSet,value):
    count=0
    for i in DataSet:
        if value in i:
            count=count+1
    return count

In [21]:
#Equations for frequency of 2 Value found together in the Data Set
def FindFrequency2 (DataSet,value,value2):
    count=0
    for i in DataSet:
        if value in i and value2 in i:
            count=count+1
    return count

In [22]:
#Equations for frequency of 3 Value found together in the Data Set
def FindFrequency3 (DataSet,value,value2,value3):
    count=0
    for i in DataSet:
        if value in i and value2 in i and value3 in i:
            count=count+1
    return count

In [23]:
# A function to sort lists by their second values
def Sort(Set):
    Set.sort(key=lambda x: x[1])
    return Set

In [24]:
def FreqValues(DataSet):
    AllValues=[]
    shape=DataSet.shape
    for i in range(shape[0]):
        for j in range(shape[1]):
            AllValues.append(DataSet[i][j])
    
    UniqueValues=[]
    for i in AllValues:
        if i not in UniqueValues:
            UniqueValues.append(i)
            
    AllFreq=[]
    for i in UniqueValues:
        AllFreq.append(FindFrequency(DataSet,i))
    
    Freq=list(zip(UniqueValues,AllFreq))
    Sort(Freq)
    return Freq

### Creating Support, Confidence and Lift

In [25]:
#Creating Support  
def CalcSupport(XY,Total):
    return XY/Total

In [26]:
#Creating Confidence
def CalcConfidence(XY,X):
    return XY/X

In [27]:
#Creating Lift
def CalcLift(supportXY,supX,supY):
    den = supX*supY
    return supportXY/den

In [28]:
#Function to find support with the FreqList from Frequency List Function
def FreqSup(Frequency,N):
    Support=[]
    for i in range(len(Frequency)):
        Support.append(CalcSupport((Frequency[i][1]),N))
    
    return Support

In [29]:
#Calculate Total 'Transactions'
TotalNoShows = 22275 
TotalShows = 88041 
Total = TotalNoShows + TotalShows
Total

110316

### Using the functions on the DataSet

In [30]:
#Using DataSet with only NoShow to find associations
NoShowFreq=FreqValues(NoShowAr)

#Using DataSet with All Values to find Associations
AllFreq=FreqValues(AllAr)

#Using DataSet with All Show to find Associations
ShowFreq=FreqValues(ShowAr)

In [31]:
#Create Frequency and Support of Show + Value patients
ShowSup=FreqSup(ShowFreq,Total)

In [32]:
#This is the Frequency and Support of these NoShow + the Values. 
NoShowSup=FreqSup(NoShowFreq,Total)

In [33]:
#This is the support of all the values on their own
AllSup=FreqSup(AllFreq,Total)

In [34]:
#Create DataFrame to merge values
Fq=pd.DataFrame(NoShowFreq[::-1],columns=['Value','NoShowFreq'])
Aq=pd.DataFrame(AllFreq[::-1], columns=['Value','TotalFreq'])
Sq=pd.DataFrame(ShowFreq[::-1], columns=['Value','ShowFreq'])

In [35]:
#Rearrange order so highest support is first
Aq['Sup'] = (AllSup[::-1])
Sq['ShowSup'] = (ShowSup[::-1])
Fq['NoShowSup'] = NoShowSup[::-1]

In [36]:
#Merge the NoShow Data Frame with information about Individual Value data
Fq=Fq.merge(Aq, on='Value')

In [37]:
#Calculate Confidence for each value
NoShowConf=[]
for i in range(len(Fq['Value'])):
    NoShowConf.append(CalcConfidence(Fq['NoShowFreq'][i],Fq['TotalFreq'][i]))
Fq['NoShowConf']= NoShowConf
Fq

,Value,NoShowFreq,NoShowSup,TotalFreq,Sup,NoShowConf
0,NoShow,22275,0.201920,22275,0.201920,1.000000
1,NoHandicap,21912,0.198629,108280,0.981544,0.202364
2,NoAlcoholism,21600,0.195801,106964,0.969615,0.201937
3,NoDiabetes,20853,0.189030,102416,0.928388,0.203611
4,NoScholarship,19698,0.178560,99473,0.901710,0.198024
...,...,...,...,...,...,...
137,-6,1,0.000009,1,0.000009,1.000000
138,17,1,0.000009,6,0.000054,0.166667
139,21,1,0.000009,1,0.000009,1.000000
140,24,1,0.000009,8,0.000073,0.125000


### Results

This is the data set sorted by descending support of No Shows + other patient descriptors

In [38]:
#Calculate Lift for each value
NoShowLift=[]
for i in range(len(Fq['Value'])):
    NoShowLift.append(CalcLift(Fq['NoShowSup'][i],Fq['NoShowSup'][0],Fq['Sup'][i]))
Fq['NoShowLift']= NoShowLift
Fq

,Value,NoShowFreq,NoShowSup,TotalFreq,Sup,NoShowConf,NoShowLift
0,NoShow,22275,0.201920,22275,0.201920,1.000000,4.952458
1,NoHandicap,21912,0.198629,108280,0.981544,0.202364,1.002200
2,NoAlcoholism,21600,0.195801,106964,0.969615,0.201937,1.000085
3,NoDiabetes,20853,0.189030,102416,0.928388,0.203611,1.008374
4,NoScholarship,19698,0.178560,99473,0.901710,0.198024,0.980703
...,...,...,...,...,...,...,...
137,-6,1,0.000009,1,0.000009,1.000000,4.952458
138,17,1,0.000009,6,0.000054,0.166667,0.825410
139,21,1,0.000009,1,0.000009,1.000000,4.952458
140,24,1,0.000009,8,0.000073,0.125000,0.619057


If NoShow Lift > 1 then association rule of value + no show is interesting in positive association <br/>
If NoShow Lift = 1 or close to 1, then association rule of value with no show is not interesting <br/>
If NoShow Lift < 1 then association rule of value + no show is interesting in negative association </br>

In [39]:
#Sort the Data Frame in descending order of lift
FqSort=Fq.sort_values(by=['NoShowLift'], ascending = False)
pd.set_option("display.max_rows", None, "display.max_columns", None)

The values that are names are neighborhoods <br/>
The values that are just numbers represent the weeks that the patient waited until their appointment <br/>
The values that shown as a range are age rages input by the patients <br/>

Results sorted by descending lift

In [40]:
FqSort

,Value,NoShowFreq,NoShowSup,TotalFreq,Sup,NoShowConf,NoShowLift
139,21,1,0.000009,1,0.000009,1.000000,4.952458
137,-6,1,0.000009,1,0.000009,1.000000,4.952458
136,19,1,0.000009,1,0.000009,1.000000,4.952458
133,ILHAS OCEÂNICAS DE TRINDADE,2,0.000018,2,0.000018,1.000000,4.952458
129,-1,4,0.000036,4,0.000036,1.000000,4.952458
0,NoShow,22275,0.201920,22275,0.201920,1.000000,4.952458
126,22,6,0.000054,10,0.000091,0.600000,2.971475
123,14,10,0.000091,22,0.000199,0.454545,2.251117
128,20,4,0.000036,9,0.000082,0.444444,2.201092
127,18,5,0.000045,13,0.000118,0.384615,1.904792


Patient features with Lifts greater than 1.3 (Positive realtionship with value and not showing up)

In [43]:
FqPos= Fq.loc[FqSort['NoShowLift'] > 1.3]
FqPos

,Value,NoShowFreq,NoShowSup,TotalFreq,Sup,NoShowConf,NoShowLift
0,NoShow,22275,0.201920,22275,0.201920,1.000000,4.952458
8,SMSRecieved,9775,0.088609,35433,0.321195,0.275873,1.366248
12,1,3662,0.033196,12007,0.108842,0.304989,1.510444
13,2,2854,0.025871,8856,0.080278,0.322267,1.596016
15,3,2254,0.020432,6693,0.060671,0.336770,1.667838
17,4,1816,0.016462,5477,0.049648,0.331568,1.642078
33,ITARARÉ,921,0.008349,3507,0.031790,0.262618,1.300603
35,5,789,0.007152,2262,0.020505,0.348806,1.727449
60,SANTOS DUMONT,369,0.003345,1274,0.011549,0.289639,1.434425
64,6,359,0.003254,1110,0.010062,0.323423,1.601741


Patientd with Features less than 0.7. (Negative relationship between value and not showing up)

In [44]:
FqNeg= Fq.loc[FqSort['NoShowLift'] < 0.7 ]
FqNeg

,Value,NoShowFreq,NoShowSup,TotalFreq,Sup,NoShowConf,NoShowLift
9,0,9541,0.086488,70601,0.639989,0.135140,0.669274
130,13,3,0.000027,23,0.000208,0.130435,0.645973
131,ILHA DO BOI,3,0.000027,35,0.000317,0.085714,0.424496
134,15,2,0.000018,24,0.000218,0.083333,0.412705
140,24,1,0.000009,8,0.000073,0.125000,0.619057
141,AEROPORTO,1,0.000009,8,0.000073,0.125000,0.619057
